<a href="https://www.kaggle.com/code/samithsachidanandan/coffee-cups-cnn-tf-improved-transfer-learning?scriptVersionId=284126235" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>



### Acknowledgements:

[Marília Prata](https://www.kaggle.com/code/mpwolke/coffee-cups-cnn-keras) Original work Published on December 04, 2025. By Prata, Marília (mpwolke)

In [ ]:
!pip install protobuf==3.20.* --force-reinstall --no-deps

In [ ]:

import numpy as np 
import pandas as pd 

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns


import plotly.io as pio
pio.renderers.default = 'iframe'

import plotly.graph_objs as go
import plotly.offline as py
import plotly.express as px


import warnings
warnings.filterwarnings('ignore')



### Coffee Grounds

"Coffee ground" can refer to either fresh, ground coffee beans before brewing or the sediment left over after brewing. The term "coffee grounds" can also refer to something that looks like coffee grounds, such as the appearance of vomit that has been darkened by blood. The more common meaning refers to the small particles of roasted coffee beans." 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization,GlobalAveragePooling2D
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.applications import MobileNetV2, EfficientNetB0


## Class_mode

https://stackoverflow.com/questions/62090714/confused-between-whether-or-not-to-change-the-class-mode-from-binary-to-categori

"So you should use categorical_crossentropy as loss function if you choose categorical for class_mode, and sparse_categorical_crossentropy if you choose sparse."

"Of course, you should change it. If you have **only 2 classes** use **class_mode='binary'** in flow_from_directory function and **binary_crossentropy** as loss. In case you have more than 2 classes, use class_mode='categorical' and categorical_crossentropy"

In [ ]:
#By Arif Miah https://www.kaggle.com/code/miadul/basic-cnn

dataset_path = "/kaggle/input/turkish-coffee-cup-grounds-image-dataset"


img_size = (224, 224)
batch = 8 


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.6, 1.4],
    fill_mode='nearest',
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)


train_data = train_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch,
    class_mode='binary',
    subset='training' 
)

val_data = train_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch,
    class_mode='binary',
    subset='validation'  
)

test_data = test_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch,
    class_mode='binary',
    shuffle=False
)

## Display images

In [ ]:
#By Arif Miah https://www.kaggle.com/code/miadul/basic-cnn

imgs, labels = next(train_data)


num_images = min(len(imgs), 9) 

plt.figure(figsize=(8,8))
for i in range(num_images):
    plt.subplot(3,3,i+1)
    plt.imshow(imgs[i])
    plt.title("No Coffee Ground" if labels[i] == 0 else "Coffee Grounds")
    plt.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
#By Arif Miah https://www.kaggle.com/code/miadul/basic-cnn

print("Classes:", train_data.class_indices)
print("Train Samples:", train_data.samples)
print("Val Samples:", val_data.samples)
print("Test Samples:", test_data.samples)

## Model

 Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None,), output.shape=(None, 2)

In [ ]:
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)


base_model.trainable = False


model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])


model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

## Epochs

No GPU cause we have just 30 images.

In [ ]:
history = model.fit(
    train_data,
    epochs=100,  
    validation_data=val_data,
    callbacks=[
        EarlyStopping(
            monitor='val_loss', 
            patience=15,  
            restore_best_weights=True,
            verbose=1  
        ),
        ReduceLROnPlateau(
            monitor='val_loss', 
            factor=0.5, 
            patience=7,  
            min_lr=1e-7,  
            verbose=1  
        ),
        ModelCheckpoint(  
            'best_model.keras',
            monitor='val_loss',
            save_best_only=True,
            verbose=1
        )
    ],
    verbose=1
)

In [ ]:
loss, acc = model.evaluate(test_data)
print("Test Accuracy:", acc)

In [ ]:
#By Arif Miah https://www.kaggle.com/code/miadul/basic-cnn

pred = model.predict(test_data)
pred_classes = (pred > 0.5).astype("int32").reshape(-1)

true_classes = test_data.classes
class_names = ["No Coffee Grounds", "Coffee Grounds"]

print(classification_report(true_classes, pred_classes, target_names=class_names))

## Confusion Matrix

In [ ]:
#By Arif Miah https://www.kaggle.com/code/miadul/basic-cnn

cm = confusion_matrix(true_classes, pred_classes)

plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

## ROC Curve

It's a right (rectangular)triangle.

In [ ]:
#By Arif Miah https://www.kaggle.com/code/miadul/basic-cnn

fpr, tpr, _ = roc_curve(true_classes, pred)
roc_auc = auc(fpr, tpr)

plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.3f}")
plt.plot([0,1], [0,1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.show()

## History

In [ ]:
#By Arif Miah https://www.kaggle.com/code/miadul/basic-cnn

plt.figure(figsize=(14,5))

# Accuracy
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Accuracy Curve")
plt.legend(["Train", "Val"])

# Loss
plt.subplot(1,2,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Loss Curve")
plt.legend(["Train", "Val"])

plt.show()

## I had to change Coffee Grounds to the 1st option to get right prediction

So, It's not a good one.

In [ ]:
test_img = "/kaggle/input/turkish-coffee-cup-grounds-image-dataset/WithCoffeeGround/IMG-20241013-WA0057_jpg.rf.58d2bc7d0343ea38bb4a69714257c3c6.jpg"

img = load_img(test_img, target_size=img_size)
img_arr = img_to_array(img)/255.
img_arr = np.expand_dims(img_arr, axis=0)

p = model.predict(img_arr)[0][0]

if p > 0.5:
    print("Prediction: Coffee Grounds")
else:
    print("Prediction: No Coffee Grounds")

In [ ]:
model.save("coffee_grounds_model.keras")
print("Keras Model Saved!")

### Acknowledgements:



[Marília Prata](https://www.kaggle.com/code/mpwolke/coffee-cups-cnn-keras)

[Arif Miah](https://www.kaggle.com/code/miadul/basic-cnn)